In [6]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import style
style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [8]:
train = pd.read_csv('/content/Google play store Review.csv')
train.head()

,content,score
0,I cannot open the app anymore,1
1,I have been begging for a refund from this app...,1
2,Very costly for the premium version (approx In...,1
3,"Used to keep me organized, but all the 1010 UP...",1
4,Dan Birthday Oct 18,1


In [9]:
def remove_pattern(text,pattern):

    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)

    return text

In [10]:
train['Tidy_Reviews'] = np.vectorize(remove_pattern)(train['content'], "@[\w]*")
train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")
train['Tidy_Reviews'] = train['Tidy_Reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train.head()

<ipython-input-10-db8fca9423f4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")


,content,score,Tidy_Reviews
0,I cannot open the app anymore,1,cannot open anymore
1,I have been begging for a refund from this app...,1,have been begging refund from this over month ...
2,Very costly for the premium version (approx In...,1,Very costly premium version approx Indian Rupe...
3,"Used to keep me organized, but all the 1010 UP...",1,Used keep organized UPDATES have made mess thi...
4,Dan Birthday Oct 18,1,Birthday


In [11]:
tokenized_review = train['Tidy_Reviews'].apply(lambda x: x.split())
tokenized_review.head()

0                              [cannot, open, anymore]
1    [have, been, begging, refund, from, this, over...
2    [Very, costly, premium, version, approx, India...
3    [Used, keep, organized, UPDATES, have, made, m...
4                                           [Birthday]
Name: Tidy_Reviews, dtype: object

In [12]:
from nltk import PorterStemmer
ps = PorterStemmer()
tokenized_review = tokenized_review.apply(lambda x: [ps.stem(i) for i in x])
tokenized_review.head()

0                               [cannot, open, anymor]
1    [have, been, beg, refund, from, thi, over, mon...
2    [veri, costli, premium, version, approx, india...
3    [use, keep, organ, updat, have, made, mess, th...
4                                           [birthday]
Name: Tidy_Reviews, dtype: object

In [13]:
for i in range(len(tokenized_review)):
    tokenized_review[i] = ' '.join(tokenized_review[i])

train['Tidy_Reviews'] = tokenized_review
train.head()

,content,score,Tidy_Reviews
0,I cannot open the app anymore,1,cannot open anymor
1,I have been begging for a refund from this app...,1,have been beg refund from thi over month nobod...
2,Very costly for the premium version (approx In...,1,veri costli premium version approx indian rupe...
3,"Used to keep me organized, but all the 1010 UP...",1,use keep organ updat have made mess thing cudn...
4,Dan Birthday Oct 18,1,birthday


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['Tidy_Reviews'])

df_bow = pd.DataFrame(bow.todense())

df_bow

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12483,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')
tfidf=tfidf.fit_transform(train['Tidy_Reviews'])
df_tfidf = pd.DataFrame(tfidf.todense())
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.234293,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12480,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
12481,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
12482,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
12483,0.0,0.0,0.0,0.468929,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [21]:
train_bow = bow[:12485]

train_bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [22]:
train_tfidf= tfidf[:12485]

train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train_bow,x_valid_bow,y_train_bow,y_valid_bow = train_test_split(train_bow,train['score'],test_size=0.3,random_state=2)
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(train_tfidf,train['score'],test_size=0.3,random_state=17)

In [25]:
pip install keras.preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 826.5 kB/s eta 0:00:00


In [26]:
from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf

In [27]:
x_train_bow = x_train_bow.toarray()
x_valid_bow = x_valid_bow.toarray()

In [28]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train_bow.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_bow, y_train_bow, batch_size=5, epochs=15)

Epoch 1/15
1748/1748 [==============================] - 18s 10ms/step - loss: 0.5524 - accuracy: 0.7192
Epoch 2/15
1748/1748 [==============================] - 20s 11ms/step - loss: 0.4639 - accuracy: 0.7999
Epoch 3/15
1748/1748 [==============================] - 11s 6ms/step - loss: 0.4463 - accuracy: 0.8083
Epoch 4/15
1748/1748 [==============================] - 12s 7ms/step - loss: 0.4361 - accuracy: 0.8147
Epoch 5/15
1748/1748 [==============================] - 5s 3ms/step - loss: 0.4289 - accuracy: 0.8176
Epoch 6/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4240 - accuracy: 0.8197
Epoch 7/15
1748/1748 [==============================] - 5s 3ms/step - loss: 0.4201 - accuracy: 0.8191
Epoch 8/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4178 - accuracy: 0.8219
Epoch 9/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4148 - accuracy: 0.8230
Epoch 10/15
1748/1748 [==============================] - 5s 3ms/step - loss:

In [29]:
score = model.evaluate(x_valid_bow, y_valid_bow, verbose=1)

118/118 [==============================] - 0s 2ms/step - loss: 0.4956 - accuracy: 0.7774


In [30]:
x_train_tfidf = x_train_tfidf.toarray()
x_valid_tfidf = x_valid_tfidf.toarray()

In [31]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train_bow.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_tfidf, y_train_tfidf, batch_size=5, epochs=15)

Epoch 1/15
1748/1748 [==============================] - 5s 3ms/step - loss: 0.5333 - accuracy: 0.7218
Epoch 2/15
1748/1748 [==============================] - 5s 3ms/step - loss: 0.4384 - accuracy: 0.7861
Epoch 3/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4232 - accuracy: 0.8003
Epoch 4/15
1748/1748 [==============================] - 5s 3ms/step - loss: 0.4149 - accuracy: 0.8054
Epoch 5/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4090 - accuracy: 0.8099
Epoch 6/15
1748/1748 [==============================] - 6s 3ms/step - loss: 0.4067 - accuracy: 0.8143
Epoch 7/15
1748/1748 [==============================] - 4s 3ms/step - loss: 0.4045 - accuracy: 0.8176
Epoch 8/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4026 - accuracy: 0.8181
Epoch 9/15
1748/1748 [==============================] - 4s 2ms/step - loss: 0.4011 - accuracy: 0.8203
Epoch 10/15
1748/1748 [==============================] - 4s 3ms/step - loss: 0.399

In [32]:
score = model.evaluate(x_valid_tfidf, y_valid_tfidf, verbose=1)

118/118 [==============================] - 2s 9ms/step - loss: 0.4859 - accuracy: 0.7803
